In [194]:
import sys
import numpy as np
import pandas as pd 
import os
import sklearn

from glob import glob
from typing import Tuple

import matplotlib.pyplot as plt

sys.path.insert(0, '../')

from pickle import load, dump
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

from src.vg2signal import read_raw_vg_as_df, make_smoother, make_shoulder_getter, make_detilter, make_signal_getter, find_first_derivative_peaks
# from src.generate_dataset import v2signal_extra_features

from src.utils  import per_error
from src.config import DATASET_PATH, models_features_per, models_features_r2

In [200]:
def v2signal_extra_features(vg_filename: str,
             do_log: bool,
             smoothing_bw: float,
             vcenter: float,
             vwidth: float,
             stiffness: float,
             noise=None,
             flip_signal=False,
             clip_signal=False,
             show_negative_peak_signal=False):

    vg_df = read_raw_vg_as_df(vg_filename)

    if (vg_df['I'].to_numpy() < 0).any():
        temp = [None] * 11
        return [None, None, vg_df] + temp

    if do_log:
        cur_var_name = "logI"
        #vg_df[cur_var_name] = np.emath.logn(logbase, vg_df["I"])
        vg_df[cur_var_name] = np.log2(vg_df["I"])
    else:
        cur_var_name = "I"

    smoother = make_smoother(smoothing_bw)

    vg_df["smoothed"] = smoother(vg_df["V"], vg_df[cur_var_name].to_numpy())

    shoulder_getter = make_shoulder_getter(1, 1.1)
    (roots_ddd, peak_v_shoulder) = shoulder_getter(vg_df["V"],
                                                     vg_df["smoothed"])

    vcenter = peak_v_shoulder
    vstart = vcenter - 0.5*vwidth
    vend   = vcenter + 0.5*vwidth

    detilter = make_detilter(vstart, vend, stiffness)
    vg_df["detilted"] = detilter(vg_df["V"].to_numpy(),
                                 vg_df["smoothed"].to_numpy())

    if flip_signal:
        if abs(vg_df['detilted'].min()) > abs(vg_df['detilted'].max()):
            print(vg_filename)
            #vg_df['detilted'] = -vg_df['detilted'] # Flip the signal if negative

    if clip_signal:
        vg_df['detilted'] = vg_df['detilted'].clip(lower=0.0)
        if vg_df['detilted'].sum()==0:
            return 0, 0, 0, vg_df, vcenter, 0, 0, 0,\
           0, 0, 0, 0, 0, 0

    if show_negative_peak_signal:
        if np.abs(vg_df['detilted'].min()) > np.abs(vg_df['detilted'].max()):
            print("Negative peak", os.path.basename(vg_filename))

    # Add Noise to the dataset
    if noise=='gaussian':
        section = vg_df["detilted"][(vg_df['V']<1.15) & (vg_df['V']>0.93)]
        print(section.std())
        section += np.random.normal(0.0005, 0.001, len(section))
        
        vg_df["detilted"][(vg_df['V']<1.15) & (vg_df['V']>0.93)] = section
    
    # print(vg_df["detilted"].shape, vg_df["smoothed"].shape)

    signal_getter = make_signal_getter(vstart, vend)
    (peak_signal_return, peak_v_return) = signal_getter(vg_df["V"], vg_df["detilted"])
    ymaxidx = np.argmax(vg_df["detilted"])

    peakarea = sklearn.metrics.auc(vg_df["V"], vg_df["detilted"])*1000

    V, dS_dV, dS_dV_max_peak, dS_dV_min_peak, dS_dV_peak_diff, \
    dS_dV_max_V, dS_dV_min_V, dS_dV_area        = find_first_derivative_peaks(vg_df["V"].values, vg_df["detilted"].values)

    signal_std  = vg_df["detilted"].values.std()
    signal_mean = vg_df["detilted"].values.mean()
    
   
    return  peakarea, peak_signal_return, peak_v_return, vg_df, vcenter, vg_df["detilted"][ymaxidx], signal_mean, signal_std,\
           dS_dV_max_peak, dS_dV_min_peak, dS_dV_peak_diff, dS_dV_max_V, dS_dV_min_V, dS_dV_area


def v2signal_raw(vg_filename: str,
             do_log: bool,
             smoothing_bw: float,
             vcenter: float,
             vwidth: float,
             stiffness: float):

    vg_df = read_raw_vg_as_df(vg_filename)

    if (vg_df['I'].to_numpy() < 0).any():
        temp = [None] * 11
        return [None, None, vg_df] + temp

    if do_log:
        cur_var_name = "logI"
        #vg_df[cur_var_name] = np.emath.logn(logbase, vg_df["I"])
        vg_df[cur_var_name] = np.log2(vg_df["I"])
    else:
        cur_var_name = "I"

    smoother = make_smoother(smoothing_bw)

    vg_df["smoothed"] = smoother(vg_df["V"], vg_df[cur_var_name].to_numpy())

    shoulder_getter = make_shoulder_getter(1, 1.1)
    (peak_signal, peak_v_shoulder) = shoulder_getter(vg_df["V"],
                                                     vg_df["smoothed"])

    vcenter = peak_v_shoulder
    vstart = vcenter - 0.5*vwidth
    vend = vcenter + 0.5*vwidth

    detilter = make_detilter(vstart, vend, stiffness)
    vg_df["detilted"] = detilter(vg_df["V"].to_numpy(),
                                 vg_df["smoothed"].to_numpy())

   
    return  vg_df

def run_vg2_raw(folderpath: str, 
            do_log:bool, 
            recenter:bool, 
            smoothing_bw:float, 
            stiffness:float, 
            vcenter:float, 
            vwidth1:float, 
            vwidth2:float,
            noise=None) -> Tuple[dict, str]:

    os.chdir(folderpath)  # change to desired folderpath

    signal_lst = []
    
    for filename in os.listdir():
        if filename[-3:] == 'txt':
            print("Analyzing:", filename)
            df = v2signal_raw(filename,
                                        do_log,
                                        smoothing_bw,
                                        vcenter,
                                        vwidth1,
                                        stiffness)

            (peak_signal, peak_curvature, peak_v, vg_df, vcenter, ph, signal_mean, signal_std,\
                dS_dV_max_peak, dS_dV_min_peak, dS_dV_peak_diff, dS_dV_max_V, \
                dS_dV_min_V, dS_dV_area) = v2signal_extra_features(filename,
                                                                    do_log,
                                                                    smoothing_bw,
                                                                    vcenter,
                                                                    vwidth1,
                                                                    stiffness,
                                                                    noise=noise)
            
            if (peak_signal == None) or (peak_curvature==None):
                #print(f"peak_signal:{peak_signal} OR peak curvature: {peak_curvature}", filename)
                continue

            idx1 = filename.rfind("cbz")
            idx2 = filename[idx1:].find("_")
            conc = filename[idx1 + 3:idx1 + idx2]
            replicate = filename[idx1 + idx2 + 1:filename.rfind(".")]

            # Crop only the signal
            try:
                crop = (df['V']<1.15) & (df['V']>0.93)
                VI   =  df['detilted'][crop].to_numpy() 
            except:
                continue

            if 'p' in conc:  
                pi = conc.find('p')
                conctemp = conc[:pi] + '.' + conc[pi + 1:]
                conc = conctemp

            signal_lst.append([filename, round(peak_signal, 4), round(peak_curvature, 4), round(peak_v, 4), round(vcenter, 4), ph, round(signal_mean, 4), round(signal_std, 4),\
                              round(dS_dV_max_peak, 4), round(dS_dV_min_peak, 4), round(dS_dV_peak_diff, 4), round(dS_dV_max_V, 4), round(dS_dV_min_V, 4), round(dS_dV_area, 4), [VI]])  # add text filename & peak signal to signal list
            
            concstrxl   = str(float(conc))

    signal_df = pd.DataFrame(signal_lst, columns=["file", "peak area", "peak curvature", "peak V", "vcenter", "PH", "signal_mean", "signal_std", \
                              "dS_dV_max_peak", "dS_dV_min_peak", "dS_dV_peak_diff", "dS_dV_max_V", "dS_dV_min_V", "dS_dV_area", 'raw'])

    
    with open(f"raw_data_feature_extracted{'_noise' if noise=='gaussian' else '' }.pickle", 'wb') as f:
        dump(signal_df, f)
   
    return    signal_df  

def load_dataset(dataset_path=None, raw=False, normalize=True) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    if dataset_path==None: dataset_path = DATASET_PATH

    if ('ML1_ML2'in os.path.basename(dataset_path)) or ('test' in os.path.basename(dataset_path)):
        datasets = sorted([f"{i}" for i in glob(f'{dataset_path}/*')])
        
        df = []
        for dataset in datasets:

            with open(dataset, 'rb') as f:
                data = load(f)
                df.append(data) 
        df = pd.concat(df)

    else:
        dataset_path = f"{dataset_path}"
        with open(dataset_path, 'rb') as f:
            df = load(f)

    if raw: X = [x.iloc[0] for (_,x) in df[['raw']].iterrows()] 
    else:   
         
        X = df.drop(columns=['raw', 'file'], inplace=False)
        X.rename(columns={"PH": 'univariate, max(S)', 'signal_std':'univariate, std(S)', 'signal_mean':'univariate, mean(S)', 'peak area':'univariate, area(S)', \
                        'dS_dV_area':'univariate, area(dS/dV)', 'dS_dV_max_peak':'univariate, max(dS/dV)', 'dS_dV_min_peak':'univariate, min(dS/dV)',\
                    'dS_dV_peak_diff':'univariate, max(dS/dV) - min(dS/dV)', \
                    'peak V':'univariate, V_max(S)', 'dS_dV_max_V':'univariate, V_max(dS/dV)', 'dS_dV_min_V':'univariate, V_min(dS/dV)',\
        }, inplace = True)
        columns = X.columns
  
    X = np.array(X).squeeze()
    y = df['file'].apply(lambda x: int(x.split('_')[-2].replace('cbz','')))

    # Split the total dataset into training (60%) and testing (40%) dataset
    X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.4, shuffle=True, random_state=20, stratify=y)

    if raw:        
        # Normalize
        if normalize:
            peak_all   = X_train.max(axis=-1)
            mean_scale = np.mean(peak_all)
            std_scale  = np.std(peak_all)
            
            X_train    = (X_train - mean_scale) / std_scale
            X_test     = (X_test  - mean_scale) / std_scale
            X          = (X - X.max(axis=-1).mean()) / X.max(axis=-1).std()
        
        X_train = pd.DataFrame(X_train)
        X_test  = pd.DataFrame(X_test)
        y_train = pd.Series(y_train)
        y_test  = pd.Series(y_test)

        X       = pd.DataFrame(X)
        y       = pd.Series(y)


    else:
        scalar = StandardScaler()
        scalar.fit(X_train)

        if normalize:
            X_train = scalar.transform(X_train)
            X_test  = scalar.transform(X_test)
            X       = scalar.fit_transform(X)
    
        X_train = pd.DataFrame(X_train, columns=columns)
        X_test  = pd.DataFrame(X_test, columns=columns)
        y_train = pd.Series(y_train)
        y_test  = pd.Series(y_test)
            
        X       = pd.DataFrame(X, columns=columns)
        y       = pd.Series(y)

    
    return (X_train, X_test, y_train, y_test), (X, y)

In [240]:
def combine_all_data(add_all_dataset=False, raw=True, include_alpha_data=False):
    if add_all_dataset:
        (ML1_X_train, ML1_X_test, ML1_y_train, ML1_y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_19_ML1/raw_data_feature_extracted.pickle', raw=raw)
        (ML2_X_train, ML2_X_test, ML2_y_train, ML2_y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_22_ML2/raw_data_feature_extracted.pickle', raw=raw)
        (ML4_X_train, ML4_X_test, ML4_y_train, ML4_y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML4/raw_data_feature_extracted.pickle', raw=raw)

        (ML1_X_train_noisy, _, ML1_y_train_noisy, _), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_19_ML1/raw_data_feature_extracted_noise.pickle', raw=raw)
        (ML2_X_train_noisy, _, ML2_y_train_noisy, _), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_22_ML2/raw_data_feature_extracted_noise.pickle', raw=raw)
        (ML4_X_train_noisy, _, ML4_y_train_noisy, _), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML4/raw_data_feature_extracted_noise.pickle', raw=raw)

            
        
        if include_alpha_data:
            (_, _, _, _), (X_alpha, y_alpha) = load_dataset(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/2023_12_12_LowConc3-alpha/raw_data_feature_extracted.pickle', raw=raw)
            (_, _, _, _), (X_beta,  y_beta)  = load_dataset(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/2023_12_15_LowConc4-beta/raw_data_feature_extracted.pickle',  raw=raw)
            (_, _, _, _), (X_gamma, y_gamma) = load_dataset(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/2023_12_17_LowConc5-gamma/raw_data_feature_extracted.pickle', raw=raw)

            (_, _, _, _), (X_alpha_noisy, y_alpha_noisy) = load_dataset(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/2023_12_12_LowConc3-alpha/raw_data_feature_extracted_noise.pickle', raw=raw)
            # (_, _, _, _), (X_beta_noisy,  y_beta_noisy)  = load_dataset(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/2023_12_15_LowConc4-beta/raw_data_feature_extracted_noisy.pickle',  raw=raw)
            # (_, _, _, _), (X_gamma_noisy, y_gamma_noisy) = load_dataset(f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/2023_12_17_LowConc5-gamma/raw_data_feature_extracted_noisy.pickle', raw=raw)

        X_train = pd.concat([ML1_X_train, ML2_X_train, ML4_X_train, 
                             ML1_X_train_noisy, ML2_X_train_noisy, ML4_X_train_noisy] + 
                            ([] if include_alpha_data else []),  axis=0)
        X_test  = pd.concat([X_beta] if include_alpha_data else [ML1_X_test,  ML2_X_test,  ML4_X_test], axis=0)
        y_train = pd.concat([ML1_y_train, ML2_y_train, ML4_y_train, 
                             ML1_y_train_noisy, ML2_y_train_noisy, ML4_y_train_noisy] + 
                            ([] if include_alpha_data else []), axis=0)
        y_test  = pd.concat([y_beta] if include_alpha_data else [ML1_y_test,  ML2_y_test,  ML4_y_test], axis=0)
        
    else:
        (X_train, X_test, y_train, y_test), _ = load_dataset('/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2', raw=raw)
        y_train, y_test = y_train.to_numpy(), y_test.to_numpy()

    return X_train, X_test, y_train, y_test
    

In [241]:
X_train, X_test, y_train, y_test = combine_all_data(add_all_dataset=True, include_alpha_data=True, raw=False)

In [242]:
X_train.shape, X_test.shape

((432, 13), (40, 13))

In [243]:
pd.DataFrame(y_train).groupby(by='file').value_counts()

file
0     147
8     143
16    142
Name: count, dtype: int64

## Feature Dataset

In [244]:
kernel  = 1**2 * RBF(length_scale=2.5)
y_LOD   = 0.9117010154341669

# R2 Score
gp_model = GaussianProcessRegressor(kernel=kernel, alpha=2)
gp_model.fit(X_train[models_features_r2['GP']], y_train)

y_pred   = gp_model.predict(X_test[models_features_r2['GP']])
score_r2 = r2_score(y_test, y_pred)

# % error
gp_model = GaussianProcessRegressor(kernel=kernel, alpha=2)
gp_model.fit(X_train[models_features_per['GP']], y_train)

y_pred   = gp_model.predict(X_test[models_features_per['GP']])
diff_err = per_error(y_test, np.abs(y_pred), y_LOD)

print("R2 Score", score_r2, "% error", diff_err)

R2 Score 0.8615477921639387 % error 15.905859928558863


## Modeling Raw Signal

In [245]:
X_train, X_test, y_train, y_test = combine_all_data(add_all_dataset=True, raw=True,  include_alpha_data=True,)

In [246]:
pd.DataFrame(y_train).groupby('file').value_counts()

file
0     147
8     143
16    142
Name: count, dtype: int64

In [247]:
kernel   = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))
gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1)

In [248]:
gp_model.fit(X_train, y_train)

GaussianProcessRegressor(alpha=1, kernel=1**2 * RBF(length_scale=1))

In [249]:
y_pred = gp_model.predict(X_test)

score_r2 = r2_score(y_test, y_pred)
diff_err = per_error(y_test, np.abs(y_pred), y_LOD)

print("R2 Score", score_r2, "% error", diff_err)

R2 Score 0.07776363126189179 % error 105.53742686474456


## Test with Alpha, Gamma, and Beta dataset

In [197]:
# Create a raw dataset for the alpha, beta and gamma dataset

for dataset_name in ['/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_19_ML1', 
                     '/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML1_ML2/2024_02_22_ML2', 
                     '/Users/sangam/Desktop/Epilepsey/Code/vgramreg/dataset/ML4']:
    
    dataset_path  = dataset_name#f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/{dataset_name}/'
    do_log        = True
    recenter      = False
    smoothing_bw  = 0.006
    stiffness     = 0
    vcenter       = 1.04
    vwidth1       = 0.15
    vwidth2       = None
    
    df =  run_vg2_raw(folderpath= dataset_path, 
                do_log=do_log, 
                recenter=recenter, 
                smoothing_bw=smoothing_bw, 
                stiffness=stiffness, 
                vcenter=vcenter, 
                vwidth1=vwidth1, 
                vwidth2=vwidth2,
                noise='gaussian')
        


Analyzing: 2024_02_19_cbz08_16.txt
0.020894227813070022
Analyzing: 2024_02_19_cbz08_02.txt
0.01733401713890756
Analyzing: 2024_02_19_cbz16_16.txt
0.04605172308607468
Analyzing: 2024_02_19_cbz16_02.txt
0.030526840414905762
Analyzing: 2024_02_19_cbz16_03.txt
0.039185586336804176
Analyzing: 2024_02_19_cbz16_17.txt
0.030746244965595547
Analyzing: 2024_02_19_cbz08_03.txt


/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp/T/ipykernel_1365/3586679293.py:62: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  vg_df["detilted"][(vg_df['V']<1.15) & (vg_df['V']>0.93)] = section
/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp

0.01576170553296216
Analyzing: 2024_02_19_cbz08_17.txt
0.02245859166712518
Analyzing: 2024_02_19_cbz08_29.txt
0.012723200499054524
Analyzing: 2024_02_19_cbz08_01.txt
0.02704802819157962
Analyzing: 2024_02_19_cbz08_15.txt
0.022831464317616505
Analyzing: 2024_02_19_cbz16_01.txt
0.023020593703589765
Analyzing: 2024_02_19_cbz16_15.txt
0.046035391313669176
Analyzing: 2024_02_19_cbz16_29.txt
0.020601006694400497
Analyzing: 2024_02_19_cbz16_28.txt
0.02176162060262001
Analyzing: 2024_02_19_cbz16_14.txt
0.026948482981780677
Analyzing: 2024_02_19_cbz08_14.txt
0.01075708565974872
Analyzing: 2024_02_19_cbz08_28.txt
0.01760641181030307
Analyzing: 2024_02_19_cbz08_04.txt
0.010620669120320603
Analyzing: 2024_02_19_cbz08_10.txt
0.02041234694802318
Analyzing: 2024_02_19_cbz16_38.txt
0.018942226793843674
Analyzing: 2024_02_19_cbz16_04.txt
0.029918757413134347
Analyzing: 2024_02_19_cbz16_10.txt


/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp/T/ipykernel_1365/3586679293.py:62: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  vg_df["detilted"][(vg_df['V']<1.15) & (vg_df['V']>0.93)] = section
/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp

0.047134992918038215
Analyzing: 2024_02_19_cbz16_11.txt
0.02631419981606719
Analyzing: 2024_02_19_cbz16_05.txt
0.03166765047551915
Analyzing: 2024_02_19_cbz16_39.txt
0.025860195389050385
Analyzing: 2024_02_19_cbz08_39.txt
0.02332487083185171
Analyzing: 2024_02_19_cbz08_11.txt
0.015890396537661028
Analyzing: 2024_02_19_cbz08_05.txt
0.024791740238095262
Analyzing: 2024_02_19_cbz08_13.txt
0.014896058782288316
Analyzing: 2024_02_19_cbz08_07.txt
0.009524557459248188
Analyzing: 2024_02_19_cbz16_13.txt
0.026474864025818497
Analyzing: 2024_02_19_cbz16_07.txt
0.0338385432207649
Analyzing: 2024_02_19_cbz16_06.txt
0.034497475178502796
Analyzing: 2024_02_19_cbz16_12.txt
0.05124095595616006
Analyzing: 2024_02_19_cbz08_06.txt
0.014236600038479388
Analyzing: 2024_02_19_cbz08_12.txt
0.024848271828695506
Analyzing: 2024_02_19_cbz00_05.txt
0.0016590967742054466
Analyzing: 2024_02_19_cbz00_11.txt
0.0031788054318662644
Analyzing: 2024_02_19_cbz00_39.txt


...

/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp/T/ipykernel_1365/3586679293.py:62: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  vg_df["detilted"][(vg_df['V']<1.15) & (vg_df['V']>0.93)] = section
/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp

Analyzing: 2024_03_08_cbz00_17.txt
0.0029263275962339427
Analyzing: 2024_03_08_cbz00_16.txt
Analyzing: 2024_03_08_cbz00_02.txt
0.005059811593229092
Analyzing: 2024_03_08_cbz00_28.txt
0.004365618879874283
Analyzing: 2024_03_08_cbz00_14.txt
0.004266052792977895
Analyzing: 2024_03_08_cbz00_01.txt
0.002868594902033819
Analyzing: 2024_03_08_cbz00_15.txt
0.002231057452016505
Analyzing: 2024_03_08_cbz00_29.txt
Analyzing: 2024_03_08_cbz16_49.txt
Analyzing: 2024_03_08_cbz00_11.txt
0.0028269667476840017
Analyzing: 2024_03_08_cbz00_05.txt
0.002225503489078806
Analyzing: 2024_03_08_cbz00_39.txt


/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp/T/ipykernel_1365/3586679293.py:62: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  vg_df["detilted"][(vg_df['V']<1.15) & (vg_df['V']>0.93)] = section
/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp

0.003726492586070796
Analyzing: 2024_03_08_cbz00_38.txt
0.002552164514913001
Analyzing: 2024_03_08_cbz00_04.txt
0.002975836919197096
Analyzing: 2024_03_08_cbz00_10.txt
0.003335137815531873
Analyzing: 2024_03_08_cbz16_48.txt
Analyzing: 2024_03_08_cbz08_48.txt
Analyzing: 2024_03_08_cbz00_06.txt
0.003770264926380774
Analyzing: 2024_03_08_cbz00_12.txt
Analyzing: 2024_03_08_cbz00_13.txt
0.004609512829880334
Analyzing: 2024_03_08_cbz00_07.txt
0.0015684326654283203


/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp/T/ipykernel_1365/3586679293.py:62: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  vg_df["detilted"][(vg_df['V']<1.15) & (vg_df['V']>0.93)] = section
/var/folders/6p/0ctyq1md3qqbfn509nf4xfgr0000gp

In [55]:
dataset_name     = '2023_12_12_LowConc3-alpha'  
dataset_path     = f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/{dataset_name}/'

(MLalpha_X_train, MLalpha_test, MLalpha_y_train, MLalpha_y_test), _ = load_dataset(dataset_path, raw=True)
        

(21, 55)


In [ ]:
# Add alpha dataset in the training dataset 
dataset_name     = '2023_12_12_LowConc3-alpha'  
dataset_path     = f'/Users/sangam/Desktop/Epilepsey/Code/vgramreg/old_dataset/{dataset_name}/feature_extraction_vwidth_0.15_root_min_testing.xlsx'
test_dataset     =  pd.read_excel(dataset_path)

MLA_X, MLA_y             =  normalizer_inference_dataset(test_dataset, normalizer_type=normalizer_type)
MLA_X_noisy, MLA_y_noisy =  normalizer_inference_dataset(test_dataset.replace('root_min_testing', 'root_min_testing_noisy'), normalizer_type=normalizer_type)
# MLA_X_train ,MLA_X_test, MLA_y_train, MLA_y_test = train_test_split(MLA_X, MLA_y, test_size=0.4, shuffle=True, random_state=42)
